# Dataset Cleanup

In [72]:
import pandas as pd

In [73]:
df_train = pd.read_csv('data/train.csv', index_col='PassengerId')
df_test = pd.read_csv('data/test.csv', index_col='PassengerId')
df_test['Survived'] = 0
df = pd.concat([df_train, df_test])
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,0,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [74]:
df.isna().sum()

Survived       0
Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [75]:
df.drop(['Name', 'Ticket'], axis=1, inplace=True)

df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Fare'].fillna(df['Fare'].mean(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Cabin'].fillna('U', inplace=True)
df['Cabin'] = df['Cabin'].apply(lambda x: x.split()[0][0])

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.000000,1,0,7.2500,U,S
2,1,1,female,38.000000,1,0,71.2833,C,C
3,1,3,female,26.000000,0,0,7.9250,U,S
4,1,1,female,35.000000,1,0,53.1000,C,S
5,0,3,male,35.000000,0,0,8.0500,U,S
...,...,...,...,...,...,...,...,...,...
1305,0,3,male,29.881138,0,0,8.0500,U,S
1306,0,1,female,39.000000,0,0,108.9000,C,C
1307,0,3,male,38.500000,0,0,7.2500,U,S


In [76]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
categorized_age = pd.cut(df['Age'], bins)
df['Categorized_age'] = categorized_age
df = df.drop(['Age'], axis=1)

cagegorized_age_columns = pd.get_dummies(df['Categorized_age'], prefix='Categorized_age')
df = pd.concat([df, cagegorized_age_columns], axis=1)
df = df.drop(['Categorized_age'], axis=1)

df

,Survived,Pclass,Sex,SibSp,Parch,Fare,Cabin,Embarked,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]","Categorized_age_(30, 40]","Categorized_age_(40, 50]","Categorized_age_(50, 60]","Categorized_age_(60, 70]","Categorized_age_(70, 80]"
PassengerId,,,,,,,,,,,,,,,,
1,0,3,male,1,0,7.2500,U,S,False,False,True,False,False,False,False,False
2,1,1,female,1,0,71.2833,C,C,False,False,False,True,False,False,False,False
3,1,3,female,0,0,7.9250,U,S,False,False,True,False,False,False,False,False
4,1,1,female,1,0,53.1000,C,S,False,False,False,True,False,False,False,False
5,0,3,male,0,0,8.0500,U,S,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,male,0,0,8.0500,U,S,False,False,True,False,False,False,False,False
1306,0,1,female,0,0,108.9000,C,C,False,False,False,True,False,False,False,False
1307,0,3,male,0,0,7.2500,U,S,False,False,False,True,False,False,False,False


In [77]:
cabin_columns = pd.get_dummies(df['Cabin'], prefix='Desk')
df = pd.concat([df, cabin_columns], axis=1)
df.drop('Cabin', axis=1, inplace=True)
df

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]",...,"Categorized_age_(70, 80]",Desk_A,Desk_B,Desk_C,Desk_D,Desk_E,Desk_F,Desk_G,Desk_T,Desk_U
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,male,1,0,7.2500,S,False,False,True,...,False,False,False,False,False,False,False,False,False,True
2,1,1,female,1,0,71.2833,C,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,1,3,female,0,0,7.9250,S,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,1,1,female,1,0,53.1000,S,False,False,False,...,False,False,False,True,False,False,False,False,False,False
5,0,3,male,0,0,8.0500,S,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,male,0,0,8.0500,S,False,False,True,...,False,False,False,False,False,False,False,False,False,True
1306,0,1,female,0,0,108.9000,C,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1307,0,3,male,0,0,7.2500,S,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [78]:
embarked_columns = pd.get_dummies(df['Embarked'], prefix='Embarked')
df = pd.concat([df, embarked_columns], axis=1)
df.drop('Embarked', axis=1, inplace=True)
df

,Survived,Pclass,Sex,SibSp,Parch,Fare,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]","Categorized_age_(30, 40]",...,Desk_C,Desk_D,Desk_E,Desk_F,Desk_G,Desk_T,Desk_U,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,male,1,0,7.2500,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
2,1,1,female,1,0,71.2833,False,False,False,True,...,True,False,False,False,False,False,False,True,False,False
3,1,3,female,0,0,7.9250,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
4,1,1,female,1,0,53.1000,False,False,False,True,...,True,False,False,False,False,False,False,False,False,True
5,0,3,male,0,0,8.0500,False,False,False,True,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,male,0,0,8.0500,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
1306,0,1,female,0,0,108.9000,False,False,False,True,...,True,False,False,False,False,False,False,True,False,False
1307,0,3,male,0,0,7.2500,False,False,False,True,...,False,False,False,False,False,False,True,False,False,True


In [79]:
gender_columns = pd.get_dummies(df['Sex'], prefix='Sex')
df = pd.concat([df, gender_columns], axis=1)
df.drop('Sex', axis=1, inplace=True)
df

,Survived,Pclass,SibSp,Parch,Fare,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]","Categorized_age_(30, 40]","Categorized_age_(40, 50]",...,Desk_E,Desk_F,Desk_G,Desk_T,Desk_U,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,1,0,7.2500,False,False,True,False,False,...,False,False,False,False,True,False,False,True,False,True
2,1,1,1,0,71.2833,False,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
3,1,3,0,0,7.9250,False,False,True,False,False,...,False,False,False,False,True,False,False,True,True,False
4,1,1,1,0,53.1000,False,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
5,0,3,0,0,8.0500,False,False,False,True,False,...,False,False,False,False,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,0,0,8.0500,False,False,True,False,False,...,False,False,False,False,True,False,False,True,False,True
1306,0,1,0,0,108.9000,False,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
1307,0,3,0,0,7.2500,False,False,False,True,False,...,False,False,False,False,True,False,False,True,False,True


In [80]:
pclass_columns = pd.get_dummies(df['Pclass'], prefix='Pclass')
df = pd.concat([df, pclass_columns], axis=1)
df.drop('Pclass', axis=1, inplace=True)
df

,Survived,SibSp,Parch,Fare,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]","Categorized_age_(30, 40]","Categorized_age_(40, 50]","Categorized_age_(50, 60]",...,Desk_T,Desk_U,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,7.2500,False,False,True,False,False,False,...,False,True,False,False,True,False,True,False,False,True
2,1,1,0,71.2833,False,False,False,True,False,False,...,False,False,True,False,False,True,False,True,False,False
3,1,0,0,7.9250,False,False,True,False,False,False,...,False,True,False,False,True,True,False,False,False,True
4,1,1,0,53.1000,False,False,False,True,False,False,...,False,False,False,False,True,True,False,True,False,False
5,0,0,0,8.0500,False,False,False,True,False,False,...,False,True,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,0,0,8.0500,False,False,True,False,False,False,...,False,True,False,False,True,False,True,False,False,True
1306,0,0,0,108.9000,False,False,False,True,False,False,...,False,False,True,False,False,True,False,True,False,False
1307,0,0,0,7.2500,False,False,False,True,False,False,...,False,True,False,False,True,False,True,False,False,True


In [81]:
df[:891].to_csv('data/train_clean.csv')
df[891:].drop(['Survived'], axis=1).to_csv('data/test_clean.csv')